In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
import pandas as pd
# df = pd.read_csv("/content/drive/MyDrive/checkpoint_12_df.csv")
df = pd.read_csv("checkpoint_12_df.csv")
df = df.astype(str)

In [2]:
df[ "references" ].describe()

count     26806
unique    20942
top          []
freq       5856
Name: references, dtype: object

In [3]:
df_empty = df[ df[ "references" ] == "[]" ]
len(df_empty)

5856

In [4]:
key = 'aa8e9c8989580dd7fb3f805ea5d42d65'

In [5]:
!pip3 install httpx
import httpx
!pip3 install demjson3
from demjson3 import decode
import json
from IPython.display import clear_output

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [31]:
current_http_code = -1
def get_scopus_citations(paper_pmid, apikey):
    headers={ "X-ELS-APIKey":apikey, "Accept":'application/json' }
    timeout = httpx.Timeout(100000.0, connect=600000.0)
    client = httpx.Client(timeout=timeout,headers=headers)
    query="&view=FULL"
    # THIS FUNCTIONS: https://api.elsevier.com/content/abstract/pubmed_id:22381459?apiKey=aa8e9c8989580dd7fb3f805ea5d42d65&view=REF
    url = f'https://api.elsevier.com/content/abstract/pubmed_id:{paper_pmid}?apiKey={apikey}&view=REF'
    r=client.get(url)
    global current_http_code
    current_http_code = r.status_code
    return r

In [33]:
row_iter = 0
total_articles = len(df_empty)
def get_article_scopus_references(row):
    global row_iter
    global total_articles
    global current_http_code
    row_iter = row_iter + 1
    print( "https://api.elsevier.com/content/abstract/pubmed_id:"+row["pmid"]+"?apiKey="+key+"&view=REF" )
    citations = get_scopus_citations( row["pmid"], key )
    json_acceptable_string = citations.text
    d = decode(json_acceptable_string)
    if ( str(current_http_code) != "200" ):
        return []
    
    print( d )
    if "abstracts-retrieval-response" not in d or d["abstracts-retrieval-response"] == None:
        return []
    if "references" not in d["abstracts-retrieval-response"]:
        return []
    if "reference" not in d["abstracts-retrieval-response"]["references"]:
        return []

    d = d["abstracts-retrieval-response"]["references"]["reference"]
    if ( d == None ):
        return []
    
    if type(d) is not list:
        d = [d]
        
    citation_jsons = []
    for citation in d:
        citation_json = {}
        if "ce:doi" in citation:
            citation_json["doi"] = citation["ce:doi"]
        if "scopus-id" in citation:
            citation_json["scopus-id"] = citation["scopus-id"]
        if "scopus-eid" in citation:
            citation_json["scopus-eid"] = citation["scopus-eid"]
        if "pubmed-id" in citation:
            citation_json["pubmed"] = citation["pubmed-id"]

        formulated_citation = ""
        if "title" in citation:
            if ( citation["title"] == None ):
                continue

        if "author-list" in citation:
            if ( citation["author-list"] != None ):
                if "author" in citation["author-list"]:
                    if type(citation["author-list"]["author"]) is list:
                        for author in citation["author-list"]["author"]:
                            formulated_citation = formulated_citation + author["ce:indexed-name"] + ", "
        formulated_citation = formulated_citation[:-2]
        if "prism:coverDate" in citation:
            if citation["prism:coverDate"] != None:
                formulated_citation = formulated_citation + " (" + citation["prism:coverDate"][:4] + ") "
        if "title" in citation:
            if citation["title"] != None:
                formulated_citation = formulated_citation + citation["title"]
        if "sourcetitle" in citation:
            if citation["sourcetitle"] != None:
                formulated_citation = formulated_citation + ", " + citation["sourcetitle"]
        if "ce:doi" in citation:
            if citation["ce:doi"] != None:
                formulated_citation = formulated_citation + " " + str(citation["ce:doi"])
        citation_json["citation"] = formulated_citation
        clear_output(wait=True)
        print( f'Computing article {row_iter} / {total_articles} ...\n\nJust got this citation json:\n{citation_json}' )
        citation_jsons.append( citation_json )
    return citation_jsons

df_empty["references"] = df_empty.apply( get_article_scopus_references, axis=1 )

Computing article 5856 / 5856 ...

Just got this citation json:
{'scopus-id': '70349257714', 'scopus-eid': '2-s2.0-70349257714', 'pubmed': '19732543', 'citation': 'Rognas L.K., Christensen E.F., Sloth E., Bendtsen T.F. (2009) Pre-hospital ultrasound, Ugeskrift for Laeger'}


<ipython-input-33-a8faeb01075d>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty["references"] = df_empty.apply( get_article_scopus_references, axis=1 )


In [34]:
df_empty.to_csv("checkpoint_scopus_citations.csv", index=False)

In [35]:
df_empty.describe()

,abstract,authors,chemicals,citation,doi,history,issn,issue,journal,keywords,...,publication_types,title,url,volume,volume_issue,xml,year,author_first,references,issn_data
count,5856,5856,5856,5856,5856,5856,5856,5856,5856,5856,...,5856,5856,5856,5856,5856,5856,5856,5856,5856,5856
unique,5854,5803,1244,5856,1896,3043,1105,241,1051,5708,...,170,5837,5856,292,2069,5856,43,5146,2855,1107
top,CURRENT DIAGNOSTIC METHODS: Helicobacter pylor...,[],{},"Malbruny B, et al. Rapid and efficient detecti...",nan,"{'pubmed': datetime.datetime(1995, 1, 1, 0, 0)...",0385-0684,3,Gan To Kagaku Ryoho,['injury'],...,{'D016428': 'Journal Article'},[Diagnosis of Helicobacter pylori infection].,https://ncbi.nlm.nih.gov/pubmed/21396219,nan,nan,"b'<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArti...",1995,Gisbert JP,[],"{""title proper"": ""gan to kagaku ry\u014dh\u014..."
freq,2,8,3812,1,3961,58,235,725,235,10,...,1699,6,1,171,171,1,271,18,3001,235
